In [2]:

import pandas as pd
import numpy
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi

data = pd.read_csv(r'C:\Users\tikun\Documents\Data analysis projects\Alcohol Consumption\nesarc_pds.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



data['S2AQ8A'] = pd.to_numeric(data['S2AQ8A'],errors='coerce')
data['S2AQ8B'] = pd.to_numeric(data['S2AQ8B'],errors='coerce')
data['S1Q10B'] = pd.to_numeric(data['S1Q10B'],errors='coerce')
data['S1Q6A'] = pd.to_numeric(data['S1Q6A'],errors='coerce')
data['AGE'] = pd.to_numeric(data['AGE'],errors='coerce')


sub1=data[(data['AGE']>=18)]

sub2 = sub1.copy()
sub2['S2AQ8A']=sub2['S2AQ8A'].replace(99, numpy.nan)
sub2['S2AQ8B']=sub2['S2AQ8B'].replace(99, numpy.nan)

C:\Users\tikun\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
sub2['Percat']=pd.qcut(sub2.S1Q10B,3,labels=["Low","Medium","High"])
sub2["Percat"] = sub2["Percat"].astype('category')

sub2['Gradecat'] = pd.qcut(sub2.S1Q6A,4,labels=["Primary","High School","College","Higher Studies"])
sub2["Gradecat"] = sub2["Gradecat"].astype('category')

recode1={1:365,2:360,3:208,4:104,5:52,6:36,7:12,8:11,9:6,10:2}
sub2['USFREQMO']=sub2['S2AQ8A'].map(recode1)
sub2['USFREQMO']=pd.to_numeric(sub2['USFREQMO'],errors='coerce')

sub2['Toxic_Lvl']=sub2['USFREQMO']*sub2['S2AQ8B']
sub2['Toxic_Lvl']=pd.to_numeric(sub2['Toxic_Lvl'],errors='coerce')

ct1 = sub2.groupby('Toxic_Lvl').size()
print (ct1)

Toxic_Lvl
2.0        2824
4.0         532
6.0        2073
8.0          52
10.0         21
11.0        818
12.0       1979
14.0          5
16.0          7
18.0        280
20.0          9
22.0        602
24.0        951
30.0         27
32.0          1
33.0        217
36.0       1309
40.0          2
42.0          1
44.0         75
48.0        170
52.0        808
54.0          2
55.0         35
60.0         55
66.0         29
72.0       1417
77.0          6
84.0         11
88.0         13
90.0          3
96.0         13
99.0          1
104.0      1537
108.0       607
110.0         3
120.0        10
121.0         1
132.0         2
143.0         1
144.0       281
156.0       579
165.0         1
168.0         1
180.0       109
192.0         1
208.0      1780
216.0       171
252.0        30
260.0       140
270.0         1
288.0        30
312.0       775
324.0         4
360.0       294
364.0        27
365.0       543
416.0      1124
432.0        16
468.0        11
504.0         1
520.0       17

In [24]:
# using ols function for calculating the F-statistic and associated p value
model1 = smf.ols(formula='Toxic_Lvl ~ C(Percat)', data=sub2)
results1 = model1.fit()
print (results1.summary())

sub3 = sub2[['Toxic_Lvl', 'Percat']].dropna()

print ('means for alcohol consumption by income status')
m1= sub3.groupby('Percat').mean()
print (m1)

print ('standard deviations for alcohol consumption by income status')
sd1 = sub3.groupby('Percat').std()
print (sd1)


                            OLS Regression Results                            
Dep. Variable:              Toxic_Lvl   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.992
Date:                Sat, 24 Oct 2020   Prob (F-statistic):             0.0502
Time:                        18:23:00   Log-Likelihood:            -2.0936e+05
No. Observations:               26642   AIC:                         4.187e+05
Df Residuals:                   26639   BIC:                         4.188e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             286.3816    

In [25]:
model1 = smf.ols(formula='Toxic_Lvl ~ C(Gradecat)', data=sub2)
results1 = model1.fit()
print (results1.summary())

sub3 = sub2[['Toxic_Lvl', 'Gradecat']].dropna()

print ('means for alcohol consumption by income status')
m1= sub3.groupby('Gradecat').mean()
print (m1)

print ('standard deviations for alcohol consumption by income status')
sd1 = sub3.groupby('Gradecat').std()
print (sd1)

                            OLS Regression Results                            
Dep. Variable:              Toxic_Lvl   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     33.23
Date:                Sat, 24 Oct 2020   Prob (F-statistic):           1.98e-21
Time:                        18:23:01   Log-Likelihood:            -2.0932e+05
No. Observations:               26642   AIC:                         4.186e+05
Df Residuals:                   26638   BIC:                         4.187e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [26]:
mc1 = multi.MultiComparison(sub3['Toxic_Lvl'], sub3['Gradecat'])
res1 = mc1.tukeyhsd()
print(res1.summary())

         Multiple Comparison of Means - Tukey HSD, FWER=0.05          
    group1         group2     meandiff p-adj   lower    upper   reject
----------------------------------------------------------------------
       College    High School  37.1407 0.0489   0.1212  74.1601   True
       College Higher Studies -28.2484  0.197  -64.973   8.4763  False
       College        Primary  64.7449  0.001  29.1343 100.3556   True
   High School Higher Studies  -65.389  0.001 -92.1584 -38.6197   True
   High School        Primary  27.6043 0.0254   2.3849  52.8236   True
Higher Studies        Primary  92.9933  0.001  68.2087 117.7779   True
----------------------------------------------------------------------
